# Databáze

In [1]:
import pandas as pd

## Instalace a import SQLAlchemy

Instalaci knihovny [SQLAlchemy](https://www.sqlalchemy.org/) provedeme přes pip (nezapomeňte mít aktivovaný venv, do kterého to chcete instalovat):

```
pip install sqlalchemy
```

In [2]:
import sqlalchemy

## Stažení filmové databáze

Pro ukázku práce s databází použijeme data od Míry Brabence.
Jde o databázi typu SQLite - skládá se z jediného souboru, ve kterém jsou všecha data.
Soubor stáhneme knihovnou Requests, kterou už máte nainstalovanou (instrukce byly v první hodině)
a uložíme ho do souboru na disku (v pracovním adresáři, tj. tam, odkud jste spustili Jupyter Notebook).

In [3]:
from pathlib import Path
import requests

In [4]:
db_data = requests.get('https://github.com/brabemi/imper-ukol/raw/master/app.db').content
Path('filmy.db').write_bytes(db_data)

995328

## Otevření databáze, průzkum struktury

In [5]:
engine = sqlalchemy.create_engine('sqlite:///filmy.db')

Objekt `engine` obsahuje připojení do databáze a skrz něj budeme provádět operace s danou databází.

Začneme tím, že se podíváme, jaká je struktura té databáze - jaké tabulky obsahuje a co obsahují ty tabulky. Každá databáze na to má jiné finty, jak toto zjistit (někdy funguje SQL příkaz `SHOW TABLES`, jindy je zase potřeba `SELECT` z metadat), naštěstí SQLAlchemy nám s tím pomůže.

In [6]:
sqlalchemy.inspect(engine).get_table_names()

['actor', 'alembic_version', 'movie', 'movie_to_actor']

In [7]:
def prozkoumat(engine):
    inspector = sqlalchemy.inspect(engine)
    for schema in inspector.get_schema_names():
        print('Schema:', schema)
        for table_name in inspector.get_table_names(schema=schema):
            print()
            print('  Table:', table_name)
            print()
            for column in inspector.get_columns(table_name, schema=schema):
                print('      Column:', column['name'].ljust(12), column['type'])

In [8]:
prozkoumat(engine)

Schema: main

  Table: actor

      Column: id           INTEGER
      Column: name         TEXT
      Column: simple_name  TEXT
      Column: csfd_id      INTEGER

  Table: alembic_version

      Column: version_num  VARCHAR(32)

  Table: movie

      Column: id           INTEGER
      Column: name         TEXT
      Column: simple_name  TEXT
      Column: csfd_id      INTEGER

  Table: movie_to_actor

      Column: movie_id     INTEGER
      Column: actor_id     INTEGER


## SELECT

In [9]:
engine.execute('SELECT id, name, csfd_id FROM actor')

In [10]:
result = engine.execute('SELECT id, name, csfd_id FROM actor LIMIT 5')
for row in result:
    print(row)

(1, 'Tim Robbins', 103)
(2, 'Morgan Freeman', 92)
(3, 'Bob Gunton', 202)
(4, 'William Sadler', 203)
(5, 'Clancy Brown', 204)


In [11]:
list(engine.execute('SELECT id, name, csfd_id FROM actor LIMIT 5'))

[(1, 'Tim Robbins', 103),
 (2, 'Morgan Freeman', 92),
 (3, 'Bob Gunton', 202),
 (4, 'William Sadler', 203),
 (5, 'Clancy Brown', 204)]

In [12]:
result = engine.execute('SELECT id, name, csfd_id FROM actor LIMIT 5')
for row in result:
    print(row[0], row[1])

1 Tim Robbins
2 Morgan Freeman
3 Bob Gunton
4 William Sadler
5 Clancy Brown


In [13]:
result = engine.execute('SELECT id, name, csfd_id FROM actor LIMIT 5')
for row in result:
    print(row['id'], row['name'])

1 Tim Robbins
2 Morgan Freeman
3 Bob Gunton
4 William Sadler
5 Clancy Brown


In [14]:
result = engine.execute('SELECT id, name, csfd_id FROM actor LIMIT 5')
for row in result:
    print(dict(row))

{'id': 1, 'name': 'Tim Robbins', 'csfd_id': 103}
{'id': 2, 'name': 'Morgan Freeman', 'csfd_id': 92}
{'id': 3, 'name': 'Bob Gunton', 'csfd_id': 202}
{'id': 4, 'name': 'William Sadler', 'csfd_id': 203}
{'id': 5, 'name': 'Clancy Brown', 'csfd_id': 204}


In [15]:
result = engine.execute('SELECT * FROM actor LIMIT 5')
for row in result:
    print(dict(row))

{'id': 1, 'name': 'Tim Robbins', 'simple_name': 'timrobbins', 'csfd_id': 103}
{'id': 2, 'name': 'Morgan Freeman', 'simple_name': 'morganfreeman', 'csfd_id': 92}
{'id': 3, 'name': 'Bob Gunton', 'simple_name': 'bobgunton', 'csfd_id': 202}
{'id': 4, 'name': 'William Sadler', 'simple_name': 'williamsadler', 'csfd_id': 203}
{'id': 5, 'name': 'Clancy Brown', 'simple_name': 'clancybrown', 'csfd_id': 204}


In [16]:
pd.read_sql_query('SELECT id, name FROM actor LIMIT 5', engine)

,id,name
0,1,Tim Robbins
1,2,Morgan Freeman
2,3,Bob Gunton
3,4,William Sadler
4,5,Clancy Brown


In [17]:
pd.read_sql_table('actor', engine)

,id,name,simple_name,csfd_id
0,1,Tim Robbins,timrobbins,103
1,2,Morgan Freeman,morganfreeman,92
2,3,Bob Gunton,bobgunton,202
3,4,William Sadler,williamsadler,203
4,5,Clancy Brown,clancybrown,204
...,...,...,...,...
7377,7378,Aaron Lustig,aaronlustig,93843
7378,7379,Alan Fudge,alanfudge,144251
7379,7380,Stuart Lancaster,stuartlancaster,374381
7380,7381,Mary Jane Heath,maryjaneheath,393710


## Průzkum dat ve filmové databázi

In [18]:
sqlalchemy.inspect(engine).get_table_names()

['actor', 'alembic_version', 'movie', 'movie_to_actor']

In [19]:
actor = pd.read_sql_table('actor', engine)
movie = pd.read_sql_table('movie', engine)
mta = pd.read_sql_table('movie_to_actor', engine)

In [20]:
actor.head()

,id,name,simple_name,csfd_id
0,1,Tim Robbins,timrobbins,103
1,2,Morgan Freeman,morganfreeman,92
2,3,Bob Gunton,bobgunton,202
3,4,William Sadler,williamsadler,203
4,5,Clancy Brown,clancybrown,204


In [21]:
movie.head()

,id,name,simple_name,csfd_id
0,1,Vykoupení z věznice Shawshank,vykoupenizvezniceshawshank,2294
1,2,Forrest Gump,forrestgump,10135
2,3,Zelená míle,zelenamile,2292
3,4,Přelet nad kukaččím hnízdem,preletnadkukaccimhnizdem,2982
4,5,Schindlerův seznam,schindleruvseznam,8653


In [22]:
mta.head()

,movie_id,actor_id
0,225,1944
1,225,1468
2,225,5830
3,225,3422
4,225,5831
